In [ ]:
%load_ext lab_black

In [ ]:
roi_id = 0
use_gpr = False

In [ ]:
"""Setup the environment."""
from pathlib import Path
import os
import sys

import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import ROIS
from brainweb_tdcs.study import get_experiments_for_roi, log_marginal_likelihood
from brainweb_tdcs.plot import (
    display_side_by_side,
    plot_bipolar_unipolar,
    plot_posterior,
)

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

# Set the random seed
RANDOM_SEED = 1234

In [ ]:
"""Set some constants."""
CAPTIONS = {
    "e": "Mean absolute magnitude of the\nelectric field",
    "e_r": "Mean absolute magnitude of the\nnormal component of electric field",
}
LABELS = {
    "e": "$| \mathbf{e} |$ (mVm$^{-1}$)",
    "e_r": "$| \mathbf{e}_r |$ (mVm$^{-1}$)",
}
FUNC_DICT = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

In [ ]:
roi = ROIS[roi_id]
experiments = get_experiments_for_roi(roi)
experiments
display(Markdown("# Compare bipolar and unipolar electrodes montages " f"({roi})"))

In [ ]:
display(
    HTML(
        f"""
<div style="background: #efefef; color: #5f5f5f; padding: 15pt;">
    <b style="display: inline-block; width: 120pt;">Region of interest</b> {roi.long_name} ({roi})</br>
    <b style="display: inline-block; width: 120pt;">Electrodes montages</b> {experiments[0].montage} (Bipolar) and {experiments[1].montage} (Unipolar)</br>
    </br>
    In this notebook, we compare the effect of a bipolar stimulation to the effect of a unipolar stimulation on the electric field computed in the region of interest resulting from the stimulations using the {'realistic distribution' if use_gpr else 'uniform distribution'}.
</div>
"""
    )
)

## Data

First, we load the data corresponding to the experiments and concatenate it inot a single data frame. We also add a `"montage"` column to keep track of which type of montage the records were acquired with (`"bipolar"` or `"unipolar"`). Then we display the values of the mean absolute magnitude of the electric field $\bar{| \pmb{e} |}$ and of its component normal to the cortical surface $\bar{| \pmb{e}_r |}$.

In [ ]:
dfs = list()
for e in experiments:
    df = e.get_data() if not use_gpr else e.get_gpr_data()
    df["montage"] = "bipolar" if e.is_bipolar else "unipolar"
    dfs.append(df[["sub", "montage", "e", "e_r"]])
data = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for i, voi in enumerate(("e", "e_r")):
    plot_bipolar_unipolar(axs[i], voi, experiments[0], data, LABELS[voi], CAPTIONS[voi])
plt.show()
columns = ["mean", "std", "min", "25%", "75%", "max"]
display_side_by_side(
    [
        data.groupby("montage").describe()["e"][columns],
        data.groupby("montage").describe()["e_r"][columns],
    ],
    ["", ""],
)

## Models

To compare the measurements acquired with the different electrodes montages, we use a Bayesian linear model:
$$
Y \sim \mathcal{N}(\mu, \sigma^2),
$$
where $Y$ is the dependent variable we focus on and
$$
\mu = \alpha + \beta_{uni} \cdot X_{uni} + \varepsilon
$$
with $\alpha$ the intercept, $\beta_{uni}$ the slope, $X_{uni}$ the independent variable and $\varepsilon$ the error term. The values of $X_{uni}$ are either $1$ if the value in `"montage"` is equal to `"unipolar"` or $0$ otherwise. This way, the bipolar population is described by $\alpha + \varepsilon$ and the value $\beta_{uni}$ corresponds to the difference between the bipolar and unipolar populations.

This model is referred to as the "*pooled model*" because it does not account for the hierarchy of the data (*i.e* some measurements belong correspond to the same subject). Hence, we also consider another model that we refer to as the "*hierarchic model*" which is defined as
$$
Y \sim \mathcal{N}(\mu_i, \sigma^2), \forall i \in [1, 20]
$$
where $i$ corresponds to the $i$-th subject and
$$
\mu_i = \alpha_i + (\beta_{uni})_i \cdot X_{uni} + \varepsilon.
$$
The definition of both the intercept and the slope thus depend on the subject and become
$$
\begin{split}
    \alpha_i &= \alpha^{(com)} + \alpha_i^{(sub)}, \\
    (\beta_{uni})_i &= \beta_{uni}^{(com)} + (\beta_{uni}^{(sub)})_i,
\end{split}
$$
with $\alpha^{(com)}$ and $\beta_{uni}^{(com)}$ respectively the common part of the intercept and of the slope and $\alpha_i^{(sub)}$ and $(\beta_{uni}^{(sub)})_i$ the subject specific part of the intercept and of the slope.

For both the pooled and the hierarchic models, weakly informative priors are set (Westfall, 2017). They are then fitted using the NUTS sampler (Hoffman *et al.*, 2011).

In [ ]:
"""Define model storages."""
models = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
fits = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
summaries = dict(
    e=dict(pooled=None, hierarchic=None), e_r=dict(pooled=None, hierarchic=None)
)
rope_vs_hdi = dict(
    e=dict(pooled=dict(), hierarchic=dict()), e_r=dict(pooled=dict(), hierarchic=dict())
)

## Pooled model

In [ ]:
"""Define models."""
for voi in ("e", "e_r"):
    models[voi]["pooled"] = bmb.Model(f"{voi} ~ montage", data)

In [ ]:
"""Fit models."""
for voi in ("e", "e_r"):
    fits[voi]["pooled"] = models[voi]["pooled"].fit(
        draws=1000, chains=4, random_seed=RANDOM_SEED
    )

In [ ]:
"""Summarize results."""
indices = ["α", "β_uni", "σ"]
for voi in ("e", "e_r"):
    summaries[voi]["pooled"] = az.summary(
        fits[voi]["pooled"], stat_funcs=FUNC_DICT, extend=False
    )
    summaries[voi]["pooled"].index = indices

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3.5))
for i, voi in enumerate(("e", "e_r")):
    rope_vs_hdi[voi]["pooled"]["β_uni"] = plot_posterior(
        axs[i],
        experiments[0],
        fits[voi]["pooled"],
        summaries[voi]["pooled"],
        "montage",
        LABELS[voi],
        CAPTIONS[voi],
        summary_param="β_uni",
        show_zero=True,
        beta_suffix="uni",
        rope_width=0.1 * data[voi].std(),
    )
fig.tight_layout()
plt.show()
display_side_by_side(
    [
        pd.concat(
            (
                summaries[voi]["pooled"],
                pd.DataFrame.from_dict(
                    rope_vs_hdi[voi]["pooled"],
                    orient="index",
                    columns=["HDI ⊂ ROPE", "ROPE ⊂ HDI"],
                ),
            ),
            axis=1,
        )
        for voi in ("e", "e_r")
    ],
    ["", ""],
)

## Hierarchic model

In [ ]:
"""Define models."""
for voi in ("e", "e_r"):
    models[voi]["hierarchic"] = bmb.Model(f"{voi} ~ montage + (montage | C(sub))", data)

In [ ]:
"""Fit models."""
for voi in ("e", "e_r"):
    fits[voi]["hierarchic"] = models[voi]["hierarchic"].fit(
        draws=1000, chains=4, random_seed=RANDOM_SEED
    )

In [ ]:
"""Summarize results."""
indices = [
    "α^(com)",
    "β^(com)_uni",
    "σ(α^(sub))",
    *[f"α^(sub)_{i}" for i in range(len(data["sub"].unique()))],
    "σ(β^(sub)_uni)",
    *[f"β^(sub)_uni_{i}" for i in range(len(data["sub"].unique()))],
    "σ",
]
for voi in ("e", "e_r"):
    summaries[voi]["hierarchic"] = az.summary(
        fits[voi]["hierarchic"], stat_funcs=FUNC_DICT, extend=False
    )
    summaries[voi]["hierarchic"].index = indices

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3.5))
for i, voi in enumerate(("e", "e_r")):
    rope_vs_hdi[voi]["hierarchic"]["β^(com)_uni"] = plot_posterior(
        axs[i],
        experiments[0],
        fits[voi]["hierarchic"],
        summaries[voi]["hierarchic"],
        "montage",
        LABELS["e"],
        CAPTIONS["e"],
        summary_param="β^(com)_uni",
        show_zero=True,
        beta_suffix="uni",
        rope_width=0.1 * data[voi].std(),
    )
fig.tight_layout()
plt.show()
rows = ["α^(com)", "σ(α^(sub))", "β^(com)_uni", "σ(β^(sub)_uni)", "σ"]
display_side_by_side(
    [
        pd.concat(
            (
                summaries[voi]["hierarchic"].loc[rows, :],
                pd.DataFrame.from_dict(
                    rope_vs_hdi[voi]["hierarchic"],
                    orient="index",
                    columns=["HDI ⊂ ROPE", "ROPE ⊂ HDI"],
                ),
            ),
            axis=1,
        )
        for voi in ("e", "e_r")
    ],
    ["", ""],
)

## References

- Hoffman *et al.* (2011) - https://arxiv.org/abs/1111.4246
- Westfall (2017) - https://arxiv.org/abs/1702.01201